In [1]:
!unzip -q data/data77571/train_and_label.zip -d data/
!unzip -q data/data77571/img_test.zip -d data/

In [2]:
import os
import numpy as np

datas = []
image_base = '/home/aistudio/data/img_train'   # 训练集原图路径
annos_base = '/home/aistudio/data/lab_train'   # 训练集标签路径

# 读取原图文件名
ids_ = [v.split('.')[0] for v in os.listdir(image_base)]

# 将训练集的图像集和标签路径写入datas中
for id_ in ids_:
    img_pt0 = os.path.join(image_base, '{}.jpg'.format(id_))
    img_pt1 = os.path.join(annos_base, '{}.png'.format(id_))
    # datas.append((img_pt0.replace('/home/aistudio', ''), img_pt1.replace('/home/aistudio', '')))
    datas.append((img_pt0, img_pt1))
    if os.path.exists(img_pt0) and os.path.exists(img_pt1):
        pass
    else:
        raise "path invalid!"

# 打印datas的长度和具体存储例子
print('total:', len(datas))
print(datas[0][0])
print(datas[0][1])
print(datas[10][:])

total: 66652
/home/aistudio/data/img_train/T064729.jpg
/home/aistudio/data/lab_train/T064729.png
('/home/aistudio/data/img_train/T004828.jpg', '/home/aistudio/data/lab_train/T004828.png')


In [3]:
import numpy as np

# 四类标签，这里用处不大，比赛评测是以0、1、2、3类来对比评测的
labels = ['建筑', '耕地', '林地',  '其他']

# 将labels写入标签文件
with open('labels.txt', 'w') as f:
    for v in labels:
        f.write(v+'\n')

# 随机打乱datas
np.random.seed(5)
np.random.shuffle(datas)

# 验证集与训练集的划分，0.05表示5%为训练集，95%为训练集
split_num = int(0.05*len(datas))

# 划分训练集和验证集
train_data = datas[:-split_num]
valid_data = datas[-split_num:]

# 写入训练集list
with open('train_list.txt', 'w') as f:
    for img, lbl in train_data:
        f.write(img + ' ' + lbl + '\n')

# 写入验证集list
with open('valid_list.txt', 'w') as f:
    for img, lbl in valid_data:
        f.write(img + ' ' + lbl + '\n')

# 打印训练集和测试集大小
print('train:', len(train_data))
print('valid:', len(valid_data))

train: 63320
valid: 3332


In [ ]:
!python PaddleSeg/train.py \
        --config PaddleSeg/configs/deeplabv3p/deeplabv3p_mobilenetv2_g.yml \
        --use_vdl \
        --do_eval \
        --save_interval 1000 \
        --save_dir output \
        --num_workers 4 \
        --learning_rate 0.08

2021-12-16 00:02:30 [INFO]	
------------Environment Information-------------
platform: Linux-4.4.0-150-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: True
NVCC: Cuda compilation tools, release 10.1, V10.1.243
cudnn: 7.6
GPUs used: 1
CUDA_VISIBLE_DEVICES: None
GPU: ['GPU 0: Tesla V100-SXM2-32GB']
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~16.04) 7.5.0
PaddlePaddle: 2.2.1
OpenCV: 4.1.1
------------------------------------------------
2021-12-16 00:02:31 [INFO]	
---------------Config Information---------------
batch_size: 240
iters: 60000
loss:
  coef:
  - 1
  types:
  - ignore_index: 255
    type: CrossEntropyLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.08
  power: 0.9
  type: PolynomialDecay
model:
  align_corners: false
  backbone:
    channel_ratio: 1.0
    min_channel: 16
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/mobilenetv2.tar.gz
    type: MobileNetV2
  backbone_indices:
  - 0
  - 3
  dspp_out_cha